In [14]:
import re
import time
import re
import os
import pandas as pd

In [155]:
path_abs = os.path.abspath(os.getcwd()).replace("\\", "/")
path_abs

'C:/Users/marze/Marzena/Priv'

# Filter exons, introns, 3-UTR, 5-UTR in reference genome UMD 3.1.gff3


In [ ]:
place = ["exon", "intron", "three_prime_UTR", "five_prime_UTR"]

t0 = time.time()
path = path_abs + "/Bos_taurus.UMD3.1.94.gff3"  #plik ze wszystkimi chromosomami: 120 MB
with open(path, "r") as f:
    for line in f:  #wczytuj po linijce
        if line[0].isdigit() or line[0] == 'X' or line[0:2] == 'MT':  #jeżeli 1 znak jest cyfrą (chromosom) / X / MT
            l = [num for num in line.split('\t')]  # dzieli linijkę na wyrazy przedzielone tabulatorem
            for i in place:
                p = bool(re.search(i, l[2]))  #logiczna zmienna: Czy jest exon w 3 kolumnie?
                if p == True:  #jeżeli znajdzie:
                    name = f'files/{i}.txt'
                    r = open(name, "a") #Zapisz w nowym pliku
                    pattern = "exon_id=ENSBTAE\d{5,12}"
                    m = (re.findall(pattern, line))
                    l[8] = str("".join(map(str, m)))
                    string = " ".join(l[0:9])
                    string = string + " \n"
                    r.write(string) #niech zapisuje tylko kolumny 1-8

t1 = time.time()
t=(t1-t0)/60
print("Time:", t, "min")  #0.5min

In [159]:
path_dupplications = [path_abs+'/duplikacje/'+file for file in os.listdir("duplikacje")] #Duplikacje
path_place = [path_abs+'/files/'+file for file in os.listdir("files")]
path_dupplications,path_place

(['C:/Users/marze/Marzena/Priv/duplikacje/BSWUSAM000000191362_dup.txt',
  'C:/Users/marze/Marzena/Priv/duplikacje/BSWUSAM000000193323_dup.txt'],
 ['C:/Users/marze/Marzena/Priv/files/exon.txt',
  'C:/Users/marze/Marzena/Priv/files/five_prime_UTR.txt',
  'C:/Users/marze/Marzena/Priv/files/three_prime_UTR.txt'])

# Filter duplications in exons, introns, 3-UTR, 5-UTR

In [ ]:
t0 = time.time()

for i in path_place:
    for j in path_dupplications:
        with open(j, "r") as d: #otwórz duplikacje/introny/UTR3/UTR5
            for lined in d: #dla każdej linijki
                ld = [num for num in lined.split(' ')] #duplikacje
                with open(i, "r") as e:   
                    for linee in e: #sprawdź każdą linijkę exonów
                        le = [num for num in linee.split(' ')] #exony
                        if ld[0] == le[0]: #jeżeli chromosom duplikacji i exonu się zgadza sprawdzaj dalej:

                            if int(le[3]) <= int(ld[1]) <= int(le[4]) and int(le[3]) <= int(ld[2]) <= int(le[4]):  #duplikacja rozpoczyna się po rozpoczęciu exonu i kończy przed zakończeniem exonu
                                name = f'Dupplications in {i.split("/")[-1].split(".")[0]} {j.split("/")[-1].split(".")[0]}.txt'
                                r = open(name, "a")
                                ld[4] = 'Inside' #4 kolumna jako opis
                                ld[3] = le[8]
                                string = " ".join(ld[0:5])+"\n"
                                print(ld[0:5])
                                r.write(string)

                            if int(le[3]) <= int(ld[1]) < int(le[4]) < int(ld[2]):    #rozpoczyna się po rozpoczęciu exonu ale kończy się po zakończeniu exonu
                                name = f'Dupplications in {i.split("/")[-1].split(".")[0]} {j.split("/")[-1].split(".")[0]}.txt'
                                ld[2] = le[4]  #Podmiana końca duplikacji
                                ld[4] = 'Cut from right side'
                                ld[3] = le[8]
                                r = open(name, "a")
                                print(ld[0:5])
                                string = " ".join(ld[0:5])+"\n"
                                r.write(string)

                            if int(ld[1]) < int(le[3]) < int(ld[2]) <= int(le[4]):  #duplikacja rozpoczyna się przed rozpoczęciem exonu i kończy się przed zakończeniem exonu
                                ld[1] = le[3]  #Podmiana początku duplikacji
                                name = f'Dupplications in {i.split("/")[-1].split(".")[0]} {j.split("/")[-1].split(".")[0]}.txt'
                                r = open(name, "a")
                                ld[4] = 'Cut from left side'
                                ld[3] = le[8]
                                string = " ".join(ld[0:5])+"\n"
                                print(ld[0:5])
                                r.write(string)

                            if int(le[3]) > int(ld[1]) and int(le[3]) > int(ld[2]) > int(le[4]): #duplikacja rozpoczyna się przed rozpoczęciem exonu i kończy po zakończeniu exonu
                                name = f'Dupplications in {i.split("/")[-1].split(".")[0]} {j.split("/")[-1].split(".")[0]}.txt'
                                ld[2] = le[4]  #Podmiana końca duplikacji
                                ld[1] = le[3] #Podmiana początku duplikacji
                                r = open(name, "a")
                                ld[4] = 'Cut from both sides'
                                ld[3] = le[8]
                                string = " ".join(ld[0:5])+"\n"
                                print(ld[0:5])
                                r.write(string)
                                
t1 = time.time()
t=(t1-t0)/60
print("Time:", t, "min")  #13 min

# SNP (Single Nucleotide Polymorphism) in duplications (in exons, introns, 3-UTR or 5-UTR)

In [ ]:
path_SNP = [path_abs+"/1.SNP i duplikacje/BSWUSAM000000193323.vcf", path_abs+"1.SNP i duplikacje/BSWUSAM000000191362.vcf"]       

with open(path_SNP[0], "r") as d: 
    for lined in d:
        line = [num for num in lined.split('\t')]
        try:
            print(line[0], line[3:5], line[7])
        except:
            pass
#         ld = [num for num in lined.split(' ')] 
#         print(ld)

In [3]:
t0 = time.time()

path_SNP = [path_abs+"/1.SNP i duplikacje/BSWUSAM000000193323.vcf", path_abs+"1.SNP i duplikacje/BSWUSAM000000191362.vcf"]       
path_place = [path_abs+"/Duplication in regions/"+file for file in os.listdir("Duplication in regions")]

for i in path_place:
    with open(i, "r") as d: #otwórz duplikacje w exonach
        for lined in d:
            ld = [num for num in lined.split(' ')] #duplikacje
            for j in path_SNP:
                with open(j, "r") as e:  #otwórz SNP
                    for linee in e: #sprawdź każdą linijkę SNP
                        snp = [num for num in linee.split('\t')] #snp rozdzielenie
                        if snp[0][3].isdigit() or snp[0] == 'X' or snp[0:2] == 'MT':  # jeżeli 1 znak jest cyfrą (chromosom) / X / MT
                            if snp[0][3:5] == ld[0]: #jeżeli chromosom duplikacji i snp się zgadza sprawdzaj dalej:
                                if int(ld[1]) <= int( snp[1]) <= int(ld[2]):  #jeżeli SNP mieści się w zakresie od początku duplikacji do jej końca:
                                    name = f'SNP in {i.split("/")[-1].split(".")[0]} {j.split("/")[-1].split(".")[0]}.txt'
                                    r = open(name, "a")
                                    string = " ".join(ld[0:4]) + " " + snp[1] + " " + " ".join(snp[3:5]) + " " + snp[7] + "\n"
#                                     print(string)
                                    r.write(string)
                                    
t1 = time.time()
t=(t1-t0)/60
print("Time:", t, "min")  #13 min

Time: 13.547259982426961 min


# Analysis

In [162]:
path_snp = [path_abs+"/SNP in duplications in regions/"+file for file in os.listdir("SNP in duplications in regions")]
path_snp

['C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in exon BSWUSAM000000191362_dup BSWUSAM000000191362.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in exon BSWUSAM000000191362_dup BSWUSAM000000193323.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in exon BSWUSAM000000193323_dup BSWUSAM000000191362.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in exon BSWUSAM000000193323_dup BSWUSAM000000193323.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in three_prime_UTR BSWUSAM000000191362_dup BSWUSAM000000191362.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in three_prime_UTR BSWUSAM000000191362_dup BSWUSAM000000193323.txt',
 'C:/Users/marze/Marzena/Priv/SNP in duplications in regions/SNP in Dupplications in three_prime_UTR BSWUSAM000000193323_dup BSWUSAM000000

In [150]:
for i in path_snp:
    df = pd.read_csv(i, header=None, sep=" ")
    df.columns = ["Chr", "Start", "End", "ID", "SNP_place", "Ref", "Alt", "DP"]
    df['DP'] = df.DP.str.extract(r'([D][P][=][0-9]*)')
    x = df.DP.str.split("=", expand=True)
    df.DP = x[1].astype(int)
    df_less = df[df.DP <= 10]
    if len(df_less) != 0:
        print(df_less)
#     print(i, df.size)

    Chr     Start       End                          ID  SNP_place Ref Alt  DP
58   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904610   A   G  10
59   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904618   C   T  10
60   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904619   C   T  10
61   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904620   A   T  10
62   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904625   T   C   9
63   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904630   G   C   9
64   29  27904501  27904742  exon_id=ENSBTAE00000480077   27904631   G   A   9


'C:/Users/marze/Marzena/Priv'